In [ ]:
import lancedb
from pathlib import Path
import pandas as pd
from copy import deepcopy
import json
import gzip
import re
import sqlite3
from sentence_transformers import SentenceTransformer

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
data_folderpath = Path('reports/vectors')
data_filename = "20250702_0720_expanded.json.gz"


index_name = 'crs_reports'
table_name = 'sections'
index_path = Path(f'../wonky_data/{index_name}')

db = 'documents.sqlite'
sql_folder_path = Path(f'../wonky_data/databases/')
sql_folder_path.mkdir(parents=True, exist_ok=True)
sql_path = sql_folder_path.joinpath(db)

table_name_metadata = 'metadata'
table_name_text = 'text'
table_name_sections = 'sections'

In [ ]:
with gzip.open(data_folderpath.joinpath(data_filename),'rt') as f:
    data = json.load(f)

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data['version_id'] = data['version_id'].astype(str)

In [ ]:
data.head()

In [ ]:
data['chunk_id'] = data.apply(lambda row: f"{row['id']}_{row['start_index']}_{row['end_index']}", axis=1)

In [ ]:
data.head()

In [ ]:
data = data.drop_duplicates(subset=['chunk_id'])

In [ ]:
data.shape

In [ ]:
metadata_df = data[['id','type','typeId','number','active','source','topics','version_id','date','title','summary','source_file']]
metadata_df['topics'] = metadata_df['topics'].str.join(',')
print(metadata_df.shape)
metadata_df = metadata_df.drop_duplicates()
print(metadata_df.shape)

text_df = data[['id','title','passage_text','start_index','end_index','chunk_id','vector']].copy()
text_df['vector'] = text_df['vector'].apply(lambda vector: ','.join([str(x) for x in vector]))
sections_df = data[['sections','start_index','end_index','chunk_id','id']].copy()

In [ ]:
sections_df.head()

In [ ]:
sections_df = sections_df.explode('sections').to_dict(orient='records')

In [ ]:
len(sections_df)

In [ ]:
sections = list()
for _section in sections_df:
    _section_data = _section['sections']
    _section_data['start_index'] = _section['start_index']
    _section_data['end_index'] = _section['end_index']
    _section_data['chunk_id'] = _section['chunk_id']
    _section_data['id'] = _section['id']
    sections.append(_section_data)

In [ ]:
_section

In [ ]:
sections_df = pd.DataFrame(sections)

In [ ]:
sections_df.head()

In [ ]:
conn = sqlite3.connect(sql_path)

In [ ]:
metadata_df.to_sql(table_name_metadata, conn, if_exists='replace')

In [ ]:
sections_df.to_sql(table_name_sections, conn, if_exists='replace')

In [ ]:
text_df.to_sql(table_name_text, conn, if_exists='replace')

In [ ]:
index = lancedb.connect(index_path)
if table_name in index.table_names():
    index.drop_table(table_name)
table = index.create_table(table_name, data=data)
table.create_index(metric='cosine')
table.create_fts_index(['id', 'typeId', 'title', 'summary', 'passage_text','chunk_id'], replace=True)

In [ ]:
encoder = SentenceTransformer('nomic-ai/nomic-embed-text-v1.5', device='mps',trust_remote_code=True)

In [ ]:
query = 'federal interest rates'
query_vector = encoder.encode(query)

In [ ]:
results = table.search(query_vector).limit(10).to_pandas()

In [ ]:
results.head()

In [ ]:
row = results.iloc[1]
chunk_id = row['chunk_id']
report_id = row['id']
start_idx = row['start_index']
end_idx = row['end_index']

In [ ]:
def get_sections_by_id(id: str, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
SELECT * FROM sections
WHERE id='{id}'""", conn)
    return lookup_df

def get_sections_by_chunk(chunk_id: str, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
SELECT * FROM sections
WHERE chunk_id='{chunk_id}'""", conn)
    return lookup_df

def get_sections_by_indexes(id: str, start_index: int, end_index: int, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
        SELECT * FROM sections
        WHERE doc_index >= {start_index} AND doc_index <= {end_index} and id = '{id}'""", conn)
    return lookup_df

In [ ]:
get_sections_by_chunk(chunk_id, conn)

In [ ]:
get_sections_by_id(report_id, conn)

In [ ]:
get_sections_by_indexes(report_id, start_idx, end_idx, conn)

In [ ]:
get_sections_by_indexes(report_id, 10, end_idx, conn)

In [ ]:
def get_text_by_id(id: str, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
    SELECT * FROM text
    WHERE id='{id}'""", conn)
    return lookup_df

def get_text_by_chunk_id(chunk_id: str, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
    SELECT * FROM text
    WHERE chunk_id='{chunk_id}'""", conn)
    return lookup_df

def get_text_by_indexes(id: str, start_index: int, end_index: int, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
    SELECT * FROM text
    WHERE id='{id}' AND start_index >= {start_index} AND end_index <= {end_index}""", conn)
    return lookup_df

def get_text_by_indexes_expansion(id: str, start_index: int, end_index: int, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
    SELECT * FROM text
    WHERE id='{id}'
    AND
    (
        (start_index <= {start_index} AND end_index >= {start_index})
        OR
        (start_index <= {end_index} AND end_index >= {end_index})
        OR
        (start_index >= {start_index} AND end_index <= {end_index})
    )
    """, conn)
    return lookup_df

def get_text_by_indexes_sections(id: str, start_index: int, end_index: int, conn: sqlite3.Connection):
    lookup_df = pd.read_sql(f"""
    SELECT * from sections
    WHERE
    id='{id}'
    AND
    doc_index >= {start_index} AND doc_index <= {end_index}""", conn)
    return lookup_df

def get_full_article_text(id: str, conn: sqlite3.Connection):
    df = get_text_by_id(id, conn)
    text = CitationFormatter().formatter_xml_tag_article(df['passage_text'].to_list(), df['chunk_id'].to_list())
    return text

def get_chunk_text_by_indexes(id: str, start_index: int, end_index: int, conn: sqlite3.Connection):
    df = get_text_by_indexes(id, start_index, end_index, conn)
    text = CitationFormatter().formatter_xml_tag_article(df['passage_text'].to_list(), df['chunk_id'].to_list())
    return text

def get_chunk_text_by_indexes_expansion(id: str, start_index: int, end_index: int, conn: sqlite3.Connection):
    df = get_text_by_indexes_expansion(id, start_index, end_index, conn)
    text = CitationFormatter().formatter_xml_tag_article(df['passage_text'].to_list(), df['chunk_id'].to_list())
    return text

def get_section_text_by_indexes(id: str, start_index, end_index, conn: sqlite3.Connection):
    df = get_text_by_indexes_sections(id, start_index, end_index, conn)
    print(df.columns)
    text = CitationFormatter().formatter_xml_tag_article(df['content'].to_list(), df['citation'].to_list())
    return text

def make_on_the_fly_citations(base_citation: str, start_index: int, number_of_records: int, connector='_'):
    return [f"""{base_citation}{connector}{_idx}""" for _idx in range(start_index, start_index+number_of_records)]

def get_section_text_by_indexes_otf_citation(id: str, start_index: int, end_index, conn: sqlite3.Connection, otf_connector='_', reset_index=True):
    df = get_text_by_indexes_sections(id, start_index, end_index, conn)
    if reset_index:
        otf_citations = make_on_the_fly_citations(base_citation=id, start_index=0, number_of_records=len(df), connector=otf_connector)
    else:
        otf_citations = make_on_the_fly_citations(base_citation=id, start_index=start_index, number_of_records=len(df), connector=otf_connector)
    text = CitationFormatter().formatter_xml_tag_article(df['content'].to_list(), otf_citations)
    return text

def get_section_text_by_indexes_otf_citation_reset(id: str, start_index: int, end_index, conn: sqlite3.Connection, otf_connector='_'):
    df = get_text_by_indexes_sections(id, start_index, end_index, conn)
    otf_citations = make_on_the_fly_citations(base_citation=id, start_index=0, number_of_records=len(df), connector=otf_connector)
    text = CitationFormatter().formatter_xml_tag_article(df['content'].to_list(), otf_citations)
    return text

class CitationFormatter():
    def formatter_boxend_section(self, text: str, citation: str):
        return f"""{text.strip()} [{citation}]"""

    def formatter_enclosed_box_section(self, text: str, citation: str):
        return f"""[{citation}]\n{text.strip()}\n[/{citation}]"""

    def formatter_xml_tag_section(self, text: str, citation: str):
        return f"""<{citation}>\n{text.strip()}\n</{citation}>"""

    def formatter_boxend_article(self, sections, citations):
        formatted_sections = list()
        for section, citation in zip(sections, citations):
            formatted_sections.append(self.formatter_boxend_section(section, citation))
        return '\n\n'.join(formatted_sections)

    def formatter_enclosed_box_article(self, sections, citations):
        formatted_sections = list()
        for section, citation in zip(sections, citations):
            formatted_sections.append(self.formatter_enclosed_box_section(section, citation))
        return '\n\n-----\n\n'.join(formatted_sections)

    def formatter_xml_tag_article(self, sections, citations):
        formatted_sections = list()
        for section, citation in zip(sections, citations):
            formatted_sections.append(self.formatter_xml_tag_section(section, citation))
        return '\n\n'.join(formatted_sections)

In [ ]:
_ = get_section_text_by_indexes_otf_citation(report_id, 8, end_idx+11, conn=conn, otf_connector='_1__', reset_index=True)

In [ ]:
print(_)